In [1]:
import pandas as pd
import os
import numpy as np
from gc import collect
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


In [2]:
directory = 'Data'
filenames = [os.path.join(directory, f) for f in os.listdir(
    directory) if os.path.isfile(os.path.join(directory, f))]


In [3]:
filenames=filenames[1:5]

In [4]:
train_df,test_df = [],[]
merged_data_open = pd.read_csv(filenames[0])
merged_data_close = pd.read_csv(filenames[1])
merged_data_high = pd.read_csv(filenames[2])
merged_data_low = pd.read_csv(filenames[3])

train_df.append(merged_data_open[:int(0.8*len(merged_data_open))])
train_df.append(merged_data_close[:int(0.8*len(merged_data_open))])
train_df.append(merged_data_high[:int(0.8*len(merged_data_open))])
train_df.append(merged_data_low[:int(0.8*len(merged_data_open))])

test_df.append(merged_data_open[int(0.8*len(merged_data_open)):])
test_df.append(merged_data_close[int(0.8*len(merged_data_open)):])
test_df.append(merged_data_high[int(0.8*len(merged_data_open)):])
test_df.append(merged_data_low[int(0.8*len(merged_data_open)):])


In [5]:
train_df[0]

,Unnamed: 0,BCO_open,BCO_open_EMA12,BCO_open_EMA26,BCO_open_Lower_Band,BCO_open_MA,BCO_open_MACD,BCO_open_MACDh,BCO_open_MACDs,BCO_open_Momentum,...,WY_open,WY_open_EMA12,WY_open_EMA26,WY_open_Lower_Band,WY_open_MA,WY_open_MACD,WY_open_MACDh,WY_open_MACDs,WY_open_Momentum,WY_open_Upper_Band
0,0,76.26,75.244875,74.115647,69.554241,73.9010,1.629840,0.065163,1.564677,0.76,...,32.24,31.897453,31.371174,29.259681,31.2845,0.781209,-0.020746,0.801955,0.31,33.309319
1,1,75.49,75.282696,74.224836,69.857992,74.1020,1.531554,-0.026498,1.558053,-0.77,...,32.39,31.973449,31.452117,29.441351,31.4100,0.763797,-0.030526,0.794323,0.15,33.378649
2,2,75.65,75.339343,74.337457,70.126206,74.2900,1.449859,-0.086554,1.536414,0.16,...,32.36,32.033064,31.523861,29.661030,31.5340,0.739058,-0.044213,0.783270,-0.03,33.406970
3,3,75.14,75.308611,74.400564,70.562205,74.4845,1.328647,-0.166213,1.494861,-0.51,...,31.97,32.023342,31.558942,30.005765,31.6520,0.680141,-0.082503,0.762645,-0.39,33.298235
4,4,75.88,75.396671,74.516369,70.880337,74.6755,1.277570,-0.173832,1.451403,0.74,...,32.67,32.123002,31.645912,30.336284,31.7950,0.682072,-0.064458,0.746530,0.70,33.253716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,4570,36.24,36.461528,37.222621,35.082707,36.8920,-0.761093,0.029970,-0.791064,0.15,...,64.83,66.346337,66.695963,64.554004,67.3465,-0.349626,-0.475766,0.126140,-0.02,70.138996
4571,4571,36.06,36.399754,37.136501,35.093023,36.7700,-0.736747,0.043454,-0.780200,-0.18,...,64.77,66.103824,66.553299,64.185350,67.1905,-0.449476,-0.460492,0.011016,-0.06,70.195650
4572,4572,36.30,36.384408,37.074538,35.063724,36.7045,-0.690131,0.072056,-0.762186,0.24,...,65.22,65.967851,66.454536,63.950686,67.0775,-0.486686,-0.398162,-0.088524,0.45,70.204314
4573,4573,36.26,36.365268,37.014202,35.065871,36.6250,-0.648934,0.090602,-0.739536,-0.04,...,64.80,65.788181,66.331978,63.662010,66.9265,-0.543797,-0.364218,-0.179579,-0.42,70.190990


In [ ]:
for i in train_df:
    i.drop(columns=['Unnamed: 0'],inplace=True)
for i in test_df:
    i.drop(columns=['Unnamed: 0'],inplace=True)

In [16]:
def create_dataset(df):
    datagens=[]
    for idx,i in enumerate(df):
        targets = np.array(i[df[idx].columns[1::10]])
        data = np.array(i)
        sequence_length = 7  # Length of each input sequence
        batch_size = 1  # Number of sequences in each batch
        output_sequence_length = 1
        generator = TimeseriesGenerator(
            data, targets, length=sequence_length, batch_size=batch_size, stride=output_sequence_length)
        datagens.append(generator)
    return datagens

In [17]:
train_datagen = create_dataset(train_df)
test_datagen = create_dataset(test_df)

In [97]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


def metrics(idx, test_df, classifier,length = len(test_df)):
    Y_true = np.array(test_df[idx][test_df[idx].columns[::10]][:length])
    Y_pred = classifier.predict(np.array(test_df[idx][:length]))
    mae = mean_absolute_error(Y_true, Y_pred)
    print(f'Mean Absolute Error: {mae}')
    mse = mean_squared_error(Y_true, Y_pred)
    print(f'Mean Squared Error (MSE): {mse}')
    r2 = r2_score(Y_true, Y_pred)
    print(f'R-squared (R²): {r2}')


In [46]:

for idx, i in enumerate(train_df):
    clf = RandomForestRegressor(
        n_estimators=100, max_depth=20, min_samples_split=25, verbose=1)
    targets = np.array(i[train_df[idx].columns[0::10]])
    data = np.array(i)
    print(idx)
    clf.fit(data,targets)
    print('train:')
    metrics(idx, train_df, clf)
    print('test:')
    metrics(idx, test_df, clf)

0
train:
Mean Absolute Error: 0.05062859969700004
R-squared (R²): 0.9999441590765703
Mean Squared Error (MSE): 0.008783869127989868
test:
Mean Absolute Error: 12.906883464137138
R-squared (R²): -3.931166106920756
Mean Squared Error (MSE): 376.6745073409462
1
train:
Mean Absolute Error: 0.05056326157810813
R-squared (R²): 0.9999335168000717
Mean Squared Error (MSE): 0.013994555155305137
test:
Mean Absolute Error: 13.209516790938903
R-squared (R²): -3.8026639541584824
Mean Squared Error (MSE): 394.99641348055735
2
train:
Mean Absolute Error: 0.0503194027577329
R-squared (R²): 0.9999448839212782
Mean Squared Error (MSE): 0.00877502693576908
test:
Mean Absolute Error: 12.52193591339195
R-squared (R²): -3.745450848606407
Mean Squared Error (MSE): 359.62584026933825
3
train:
Mean Absolute Error: 0.051258036055694615
R-squared (R²): 0.9999419528004044
Mean Squared Error (MSE): 0.009597302956951319
test:
Mean Absolute Error: 12.869539542886066
R-squared (R²): -3.8976598072193043
Mean Squared E

In [91]:
idx=0
i = train_df[idx]
clf = RandomForestRegressor(n_jobs=10)
targets = np.array(i[train_df[idx].columns[::10]])
data = np.array(i)
print(idx)
clf.fit(data, targets)


0


RandomForestRegressor(n_jobs=10)

In [103]:

print('train:')
metrics(idx, train_df, clf,-100)
print('test:')
metrics(idx, test_df, clf,20)


train:
Mean Absolute Error: 0.1620907168207211
Mean Squared Error (MSE): 0.08066640310676781
R-squared (R²): 0.9994778317520044
test:
Mean Absolute Error: 1.862964318181823
Mean Squared Error (MSE): 8.964277400568202
R-squared (R²): -6.189068496154001


In [88]:
# Extract feature importance
feature_importance = clf.feature_importances_

# Create a list of feature names or column names
feature_names = list(train_df[0].columns)  # Replace with your actual feature names

# Create a dictionary mapping feature names to their importance values
feature_importance_dict = dict(zip(feature_names, feature_importance))

# Sort the features by importance in descending order
sorted_features = sorted(feature_importance_dict.items(),
                         key=lambda x: x[1], reverse=True)

# Print the sorted features
for feature, importance in sorted_features:
    print(f"Feature: {feature}, Importance: {importance}")


Feature: WY_open_Lower_Band, Importance: 0.1967756423308621
Feature: UGP_open_EMA26, Importance: 0.11997604645500798
Feature: PAG_open, Importance: 0.07707590981296344
Feature: UGP_open, Importance: 0.06515936856320936
Feature: UGP_open_Upper_Band, Importance: 0.05520531361018499
Feature: CAL_open_EMA12, Importance: 0.041535632041748365
Feature: UGP_open_EMA12, Importance: 0.03592652366855288
Feature: CAL_open_EMA26, Importance: 0.02871393111651284
Feature: PAG_open_EMA26, Importance: 0.028657955605881573
Feature: UGP_open_MA, Importance: 0.02864510200049887
Feature: UFI_open_EMA26, Importance: 0.026790381628748593
Feature: CAL_open_MA, Importance: 0.025607960022113435
Feature: CAL_open_Lower_Band, Importance: 0.024507641203730925
Feature: BCO_open, Importance: 0.012270007677835008
Feature: BCO_open_EMA26, Importance: 0.01170499160733269
Feature: UFI_open_Lower_Band, Importance: 0.009554754951737961
Feature: IBN_open_Upper_Band, Importance: 0.008963889771026582
Feature: IBN_open_EMA26,